# PRACTICAL TASK 3

In [214]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

## TASK 1

In [215]:
itemset = ["ABCD", "ACDF", "ACDEG", "ABDF", "BCG", "DFG", "ABG", "CDFG"]
tid = ['t' + str(n) for n in range(1, 9)]

In [216]:
df = pd.DataFrame([tid, itemset]).T
df.head()

,0,1
0,t1,ABCD
1,t2,ACDF
2,t3,ACDEG
3,t4,ABDF
4,t5,BCG


In [217]:
# Apriori
support = 3/8

association_rules = apriori(df, min_support=support, use_colnames=True)

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value t1

In [ ]:
# FPGRowth